In [11]:
import json
from urllib.request import urlopen
import pandas as pd
import numpy as np
import spacy
import matplotlib.pyplot as plt
from itertools import islice
from sklearn import preprocessing
import spacy
from spacy import displacy
import sklearn.metrics
import plotly.express as px
import pickle
# import wordcloud
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [12]:
TRAIN_DATA = "https://storage.googleapis.com/indianlegalbert/OPEN_SOURCED_FILES/Rhetorical_Role_Benchmark/Data/train.json"
train_json = json.loads(urlopen(TRAIN_DATA).read())
VAL_DATA = "https://storage.googleapis.com/indianlegalbert/OPEN_SOURCED_FILES/Rhetorical_Role_Benchmark/Data/dev.json"
val_json = json.loads(urlopen(VAL_DATA).read())

In [13]:
def preprocess_json(t_json):
    docs = []
    for rec in t_json:
        doc = []
        for line in rec['annotations'][0]['result']:
            doc.append({'text': line['value']['text'], 'label': line['value']['labels'][0] 
            if len(line['value']['labels'])>0 else 'NONE'})
        docs.append(doc)
    docs_df = pd.json_normalize([item for sublist in docs for item in sublist])

    le = preprocessing.LabelEncoder()
    le.fit(docs_df.label)
    sd = le.transform(docs_df.label)
    docs_df.label = sd
    return docs_df, docs
# print(docs[9][0])
t_docs_df, t_docs = preprocess_json(train_json)
v_docs_df, v_docs = preprocess_json(val_json)

In [14]:
def multi_acc(y_pred, y_test): 
    print(len(y_pred), len(y_test))   
    correct_pred = (y_pred == y_test)
    print(correct_pred)
    acc = correct_pred.sum() * 1.0 / len(correct_pred)
    acc = np.round_(acc * 100, decimals = 3)
    return acc

In [15]:
xtrain, xval = list(t_docs_df['text']), list(v_docs_df['text'])
ytrain, yval = np.array(t_docs_df['label']), np.array(v_docs_df['label'])

In [16]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 5), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(xtrain+xval)
xtrain_tfv = tfv.transform(xtrain) 
xval_tfv = tfv.transform(xval)

In [17]:
clf = LogisticRegression(C=1.0, solver='liblinear', max_iter=1000)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xval_tfv)
y_pred = [np.argmax(p) for p in predictions]
# print(ytrain, y_pred)
# print ("logloss: %0.3f " % multiclass_logloss(ytrain, predictions))


In [19]:
len(predictions), len(predictions[0])

(2879, 13)

In [20]:
file = open('model_preds/tfidf_val_predss.pkl', 'wb')
pickle.dump(predictions, file)
file.close()

In [26]:
file = open('model_preds/deberta_val_preds.pkl', 'rb')
data = pickle.load(file)
file.close()

In [27]:
len(data), len(data[0])

(360, 8)

In [28]:
data.shape

AttributeError: 'list' object has no attribute 'shape'